# Hands-On Text Models and Logistic Regression 
***

In this notebook we'll explore Scikit-Learn's ability to create features from text, including how to use the Bag-of-Words model.  We'll also explore a more sophisticated feature creation technique called TFIDF.  Finally, we'll use Scikit-Learn's Logistic Regression classifier to make semantic predictions about movie reviews. 

In [1]:
import numpy as np
import matplotlib.pylab as plt 
%matplotlib inline 

### Part 1: The Bag-of-Words Text Model 
***

In the first portion of this notebook we'll explore the bag of words text model.  In lecture we worked with the following corpus of documents related to newspaper titles: 

$\texttt{Training Set}:$

$\texttt{d1}: \texttt{new york new tribune}$

$\texttt{d2}: \texttt{new york times}$

$\texttt{d3}: \texttt{los angeles times}$

First we'll define the vocabulary based on the words in the test set.  It is $V = \{ \texttt{angeles}, \texttt{los}, \texttt{new}, \texttt{times}, \texttt{tribune}, \texttt{york}\}$. 

We need to define an association between the particular words in the vocabulary and the specific entries in our vectors.  Let's define this association in the order that we've listed them above.  We can store this mapping as a Python dictionary as follows: 

In [4]:
V = {'angeles': 0, 'los': 1, 'new': 2, 'times': 3, 'tribune': 4, 'york': 5}

Let's also store the documents in a list as follows: 

In [5]:
D = ["new york new tribune", "new york times", "los angeles times"]

To be consistent with sklearn conventions, we'll encode the documents as *row-vectors* stored in a **Document-Term Matrix**.  In this case, each row of the matrix corresponds to a document, and each column corresponds to a term in the vocabulary.  For our example this gives us a matrix `Xdt` of shape $3 \times 6$.  The $(d,t)$-entry in `Xdt` is then the number of times the term $t$ appears in document $d$

**Part A**: Your first task is to write some simple Python code to construct the document-term matrix `Xdt`

In [6]:
Xdt = np.zeros((len(D),len(V)))

for ii, doc in enumerate(D): 
    for term in doc.split(): 
        Xdt[ii, V[term]] += 1
        
print(Xdt)

[[ 0.  0.  2.  0.  1.  1.]
 [ 0.  0.  1.  1.  0.  1.]
 [ 1.  1.  0.  1.  0.  0.]]


Hopefully your code returns the matrix 

$$\texttt{Xdt} = 
\left[
\begin{array}{ccccccc}
0 & 0 & 2 & 0 & 1 & 1 \\
0 & 0 & 1 & 1 & 0 & 1 \\
1 & 1 & 0 & 1 & 0 & 0 \\
\end{array}
\right]$$.  

Note that the entry in the (2,0) position is $1$ because the first word (angeles) appears once in the third document. Similarly the entry in the (0,2) position is $2$ because the third word (new) appears twice in the first document. 

OK, let's see how we can construct the same document-term matrix in sklearn.  We will use something called the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html">CountVectorizer</a> to accomplish this. Let's see some code and then we'll explain how it functions. 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer    # import CountVectorizer 
bagofwords = CountVectorizer()                                 # initialize the vectorizer
Xbw = bagofwords.fit_transform(D)                              # fit to training data and transform to matrix 

The $\texttt{fit_transform}$ method actually does two things.  It fits the model to the training data by building a vocabulary.  It then transforms the text in $D$ into matrix form.  

If we wish to see the vocabulary you can do it like so 

In [8]:
print(bagofwords.vocabulary_)

{'new': 2, 'york': 5, 'tribune': 4, 'times': 3, 'los': 1, 'angeles': 0}


Note that this is the same vocabulary and indexing that we defined ourselves.  Hopefully that means we'll get the same term-frequency matrix.  We can print `Xbw` and check 

In [9]:
print(Xbw.todense())

[[0 0 2 0 1 1]
 [0 0 1 1 0 1]
 [1 1 0 1 0 0]]


Yep, they're the same!  Notice that we had to convert $X$ to a dense matrix for printing.  This is because CountVectorizer actually returns a sparse matrix.  This is a very good thing since most vectors in a text model will be **extremely** sparse, since most documents will only contain a handful of words from the vocabulary. 


OK, now suppose that we have a query document not included in the training set that we want to vectorize.  

In [10]:
d4 = ["new york post"]

We've already fit the CountVectorizer to the training set, so all we need to do is transform the test set documents into a term-frequency vector using the same conventions.  Since we've already fit the model, we do the transformation with the $\texttt{transform}$ method: 

In [11]:
x4 = bagofwords.transform(d4)

Let's print it and see what it looks like 

In [12]:
print(x4.todense())

[[0 0 1 0 0 1]]


**Part B**: What's missing from `x4` that we might expect to see from the query document? 

 The word $\texttt{post}$ does not appear in vector `x4` at all.  This is because it did not occur in the training set, which means it is not present in the vocabulary.  This should not bother us too much.  Most reasonable text data sets will have most of the important words present in the training set and thus in the vocabulary.  On the other hand, the throw-away words that are present only in the test set are probably useless anyway, since the learning model is trained based on the text in the training set, and thus won't be able to do anything intelligent with words the model hasn't seen yet. 

<br>

### Part 2: Classifying Semantics in Movie Reviews
***
> The data for this problem was taken from the <a href="https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words">Bag of Words Meets Bag of Popcorn</a> Kaggle competition

In this problem you will use the text from movie reviews to predict whether the reviewer felt positively or negatively about the movie using a Bag-of-Words text model. I've partially cleaned the data and stored training and validation sets in `data/movieReviews/movieTrainData.tsv` and `data/movieReviews/movieValidationData.tsv`.  We'll load the data and process it a bit more in the following cell. 

In [13]:
import pandas as pd

def read_and_clean_data(fname, remove_stops=True):
    
    # Read in text and labels from file 
    df = pd.read_csv(fname, sep='\t', header=None)
    df.columns = ["labels", "text"]
    
    # Make everything lowercase
    df.loc[:,"text"] = df["text"].apply(lambda s: s.lower())
    
    return df["text"].values, df["labels"].values

In [27]:
text_train, labels_train = read_and_clean_data("data/movieReviews/movieTrainData.tsv")
text_valid, labels_valid = read_and_clean_data("data/movieReviews/movieValidationData.tsv")

**Part A**: Look at a few of the reviews stored in $\texttt{text_train}$ as well as their associated labels in $\texttt{labels_train}$.  Can you figure out which label refers to a positive review and which refers to a negative review? 

In [28]:
ind = 0 
print("text labeled {}\n\n".format(labels_train[ind]), text_train[ind])

text labeled 1

 this film could be one of the most underrated film of bollywood history this blockbuster had all of it good performances music and direction i remember i was in allahabad when this movie was running and it was somewhere in march at holi time the people there were playing its song ooe amma at their loudspeakers in highest volume if someone who likes to watch some like it hot and drools over marilyn monroe he should see this movie thumbs up to govinda how many of you know that this film was shot in south of india and after sholay could be one of the very few blockbuter to hit silver screen with films like these indian comedy could never be dead


In [29]:
ind = 1 
print("text labeled {}\n\n".format(labels_train[ind]), text_train[ind])

text labeled 1

 four stories written by robert bloch about various people who live in a beautiful old mansion and what happens to them the first has denholm elliott as a novelist who sees the killer he s writing about come to life some spooky moments and the twist at the end was good the second has peter cushing becoming obsessed with a wax figure resembling his dead wife the third has christopher lee who has a child chloe franks and is scared of her it all leads up to a pretty scary ending although the ending in the story was much worse the last is an out and out comedy with jon petwee and ingrid pitt both chewing the scenery and a cape that turns people into vampires there s also a cute line about christopher lee playing dracula this is a good horror anthology nothing terrifying but the first one and the ending of the third gave me a few pleasurable little chills also the fourth one is actually very funny and pitt makes a very sexy vampire also the house itself looks beautiful and v

From this it looks like the label $y=1$ corresponds to a positive review and $y=0$ corresponds to a negative review. 

OK, the first thing we'll do is train a logistic regression classifier using the Bag-of-Words model, and see what kind of accuracy we can get.  To get started, we need to vectorize the text into numerical features that we can use.  We'll use CountVectorizer to turn our text into features, Scikit-Learn's [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) classifier to do classification, and a pipeline to tie everything together. First though, we'll fit our CountVectorizer independently so we can learn more about the data. 

In [30]:
bagofwords = CountVectorizer()
Xbw_train = bagofwords.fit_transform(text_train)

**Part B**: How many different terms are in the vocabulary? 

In [31]:
print("There are {:d} terms in the vocabular".format(Xbw_train.shape[1]))

There are 17948 terms in the vocabular


**Part C**: What are the most common words in the corpus? 

In [32]:
def most_common_words(X, vocab, num_common=20):
    word_freqs = X.sum(axis=0)
    sorted_indices = np.argsort(word_freqs)[0,::-1]
    print("word         freq")
    print("-----------------")
    for ii in range(num_common):
        word_ind = sorted_indices[0,ii]
        word =list(vocab.keys())[list(vocab.values()).index(word_ind)] 
        word_freq = word_freqs[0,word_ind]
        print("{:12s} {:d}".format(word, word_freq))

most_common_words(Xbw_train, bagofwords.vocabulary_)

word         freq
-----------------
the          13739
and          6568
of           5918
to           5555
is           4351
it           3996
in           3863
this         3037
that         2995
was          1912
as           1853
for          1846
with         1820
movie        1787
but          1692
film         1621
you          1433
on           1385
not          1239
he           1227


It looks like we have some overly common stop words that are probably not useful for classification.  We can remove them by passing in the `stop_words` parameter to the `CountVectorizer`.  We need to tell it which language we're using. How many words are left after removing the stop words? 

**Part D**: Next we'll build a pipeline that turns text into features using CountVectorizer and then performs classification using LogisticRegression. 

In [33]:
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LogisticRegression
bagofwords_steps = [
    ("bagofwords", CountVectorizer(stop_words="english")),
    ("logreg", LogisticRegression())
]
bagofwords_pipe = Pipeline(bagofwords_steps)

We can fit our model to the training data using the `.fit()` method. 

In [34]:
bagofwords_pipe.fit(text_train, labels_train)

Pipeline(memory=None,
     steps=[('bagofwords', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

Now let's make some predictions.  The following code will print a message form the validation set, along with it's true label, and then make a prediction using the pipeline. Test it out for a few validation examples by changing the value of `ind`.  How does the classifier seem to be doing? 

In [35]:
ind = 1 
print("text labeled {:d}:\n".format(labels_valid[ind]))
print(text_valid[ind])
pred = bagofwords_pipe.predict([text_valid[ind]])
print("\npredicted label: {:d}".format(pred[0]))

text labeled 0:

if you enjoy the original snl cast and shows then avoid this movie at all costs when this first came out my friends and i waited in line for over an hour to get in to a sold out movie house half way through the movie the theatre was empty we refused to leave thinking it would get better when the movie ended we were the only ones left in the theatre the movie lasted only one day in all theaters then vanished from sight in interviews with mr mike he refused to comment on this film the film was an inside joke on the episodes of snl that came out right after the films release and closing in one day we all tried to contact mr mike by phone and mail to get a refund but were totally ignored

predicted label: 0


**Part D**: Finally, let's make predictions on the the entirety of both the training and the validation sets and compute overall accuracies.  We could do this by hand, but LogisticRegression (like most models in Scikit-Learn) includes a `.score()` method which takes in data and the true labels, makes predictions, and then computes the accuracy.  Since LogisticRegression is part of our pipeline, our pipeline inherits this score function. 

In [36]:
train_acc = bagofwords_pipe.score(text_train, labels_train)
valid_acc = bagofwords_pipe.score(text_valid, labels_valid)
print("Accuracy on Training Set:   {:.3f}".format(train_acc))
print("Accuracy on Validation Set: {:.3f}".format(valid_acc))

Accuracy on Training Set:   1.000
Accuracy on Validation Set: 0.787


It looks like we're getting $100\%$ accuracy on the training set (which should worry you) and around a $79\%$ on the validation set.  Clearly, our features are overfitting the training data.  Let's see if regularization would be helpful.  The `LogisticRegression` method takes a parameter `C` for regularization strength.  The default setting is `C=1`. Try increasing this value and see if the validation accuracy improves. 

### Part 3: Term Frequency - Inverse Document Frequency 
***

The Bag-of-Words model for text classification is very popular, but let's see if we can do better.  Currently we're weighting every word in the corpus by it's frequency.  It turns out that in text classification there are often features that are not particularly useful predictors for the document class, either because they are too common or too uncommon.  **Stop-words** are extremely common, low-information words like "a", "the", "as", etc.  Removing these from documents is typically the first thing done in peparing data for document classification. 



Other words that tend to be uninformative predictors are words that appear very very rarely.  In particular, if they do not appear frequently enough in the training data then it is difficult for a classification algorithm to weight them heavily in the classification process. 

In general, the words that tend to be useful predictors are the words that appear frequently, but not too frequently.  Consider the following frequency graph for a corpus. 

<img src="figs/feat_freq.png" width=400 height=50>

The features in column A appear too frequently to be very useful, and the features in column C appear too rarely. One first-pass method of feature selection in text classification would be to discard the words from columns A and C, and build a classifier with only features from column B.

Another common model for identifying the useful terms in a document is the Term Frequency - Inverse Document Frequency (tf-idf) model.  Here we won't throw away any terms, but we'll replace their Bag-of-Words frequency counts with tf-idf scores which we describe below. 

The tf-idf score is the product of two statistics, *term frequency* and *inverse document frequency*


$$\texttt{tfidf(d,t)} = \texttt{tf(d,t)} \times \texttt{idf(t)}$$

The term frequency $\texttt{tf(d,t)}$ is a measure of the frequency with which term $t$ appears in document $d$.  The inverse document frequency $\texttt{idf(t)}$ is a measure of how much information the word provides, that is, whether the term is common or rare across all documents.  By multiplying the two quantities together, we obtain a representation of term $t$ in document $d$ that weighs how common the term is in the document with how common the word is in the entire corpus. You can imagine that the words that get the highest associated values are terms that appear many times in a small number of documents. 


There are many ways to compute the composite terms $\texttt{tf}$ and $\texttt{idf}$.  For simplicity, we'll define $\texttt{tf(d,t)}$ to be the number of times term $t$ appears in document $d$ (i.e., Bag-of-Words). We will define the inverse document frequency as follows: 

$$
\texttt{idf(t)} = \log \left(~ \frac{1 + \textrm{total # documents}}{1 + \textrm{# documents with term }t}\right) + 1
 = \log \left( \frac{1+n_d}{1+\texttt{df}(t)} \right) + 1
$$


**Part A**: Compute $\texttt{idf(t)}$ for the words `tribune` and `new` in our toy training set. 

$\texttt{Training Set}:$

$\texttt{d1}: \texttt{new york new tribune}$

$\texttt{d2}: \texttt{new york times}$

$\texttt{d3}: \texttt{los angeles times}$


We have three documents in the training set, so we set $n_d = 3$.  We need to compute $\texttt{df}(t)$ for $t=$ `tribune` and $t=$ `new`. We have  

$$
\texttt{df(tribune)} = 1 \quad \textrm{and} \quad
\texttt{df(new)} = 2 
$$

Plugging these values into our formula gives 

$$
\texttt{idf(tribune)} = \log\left(\frac{1+3}{1+1} \right) + 1 = \log\left(2 \right) + 1 \approx 1.69 
$$

and 

$$
\texttt{idf(new)} = \log\left(\frac{1+3}{1+2} \right) + 1 = \log\left(\frac{4}{3} \right) + 1 \approx 1.28 
$$

Notice that the `idf` score for tribune is higher than the `idf` score for new, because `tribune` appears in fewer documents than the word `new`.  This means that when we compute `tfidf` features, the word `tribune` gets an extra bump over the word `new`.  

**Part B**: Compute the `tfidf` feature values for the word `new` in the first two documents, as well as the word `tribune` in the first document. 


\begin{eqnarray}
\nonumber \texttt{tfidf(d1, new) = tf(d1,new)} \times \texttt{idf(new)} &=& 2 \times 1.28 = 2.56 \\
\nonumber \texttt{tfidf(d2, new) = tf(d2,new)} \times \texttt{idf(new)} &=& 1 \times 1.28 = 1.28 \\
\nonumber \texttt{tfidf(d1, tribune) = tf(d1,tribune)} \times \texttt{idf(tribune)} &=& 1 \times 1.69 = 1.69 \\
\end{eqnarray}

Note that if we compare the feature values for `new` across documents, we see that `new` is given twice the weight in `d1` compared to `d2`, which makes sense because `new` appears twice as frequently in `d1`.  However, despite the fact that `new` appears twice as frequently in `d1` as `tribune`, it does not have twice as much weight due to the `idf` score for `tribune` giving it that extra bump. 

**Part C**: OK, let's construct TFIDF features for our movie reviews, and see if we get a better LogisticRegression classifier.  All we need to do is replace `CountVectorizer` with [`TfidfVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) in our pipeline. If you go to the documentation, you'll notice that `TfidfVectorizer` includes many of the same parameters as `CountVectorizer`, including the ability to automatically remove stop words.  

In [37]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer   

# Build the pipeline 
tfidf_steps = [
    ("tfidf", TfidfVectorizer(stop_words="english")),
    ("logreg", LogisticRegression(C=1))
]
tfidf_pipe = Pipeline(tfidf_steps)

# Fit model 
tfidf_pipe.fit(text_train, labels_train)

# Get accuracies on training and validation set 
train_acc = tfidf_pipe.score(text_train, labels_train)
valid_acc = tfidf_pipe.score(text_valid, labels_valid)
print("Accuracy on Training Set:   {:.3f}".format(train_acc))
print("Accuracy on Validation Set: {:.3f}".format(valid_acc))

Accuracy on Training Set:   0.993
Accuracy on Validation Set: 0.830


**Part D**: How'd you do?  Did the TFIDF features perform better on the validation set than regular old Bag-of-Words?  What happened with the training set? 

It looks like we get about a $4\%$ bump in validation accuracy by using the TFIDF features.  Additionally, the accuracy on the training set went down, indicating that we're not overfitting as much with the TFIDF features. 

<br><br><br><br><br><br><br>
<br><br><br><br><br><br><br>
<br><br><br><br><br><br><br>
<br><br><br><br><br><br><br>

In [202]:
from IPython.core.display import HTML
HTML("""
<style>
.MathJax nobr>span.math>span{border-left-width:0 !important};
</style>
""")